### Functions

In [1]:
# choose execl 
def choose_file():
    
    return 'FFG Holdings+Opt 05-11-18 NEW' + '.xlsx'
    #return 'SCT 04-30-18' + '.xlsx'

In [2]:
# input shares dates
def get_date():
    
    # !IMPORTANT! add 1 day to period, because `get` returns previous day
    return datetime.date(2018, 05, 10) + timedelta(days=1)  
    #return datetime.datetime.now()


In [3]:
# read xls into pandas table
def create_table(file_name):
    tables_list = pd.read_excel( file_name, sheetname=None)
    
    # get last worksheet from list
    table = tables_list[list(tables_list.keys())[-1]]
    
    # clear nan rows
    table = table[pd.isnull(table.SYMBOL) == False]
    
    # extract existing found-codes
    symbl = table.SYMBOL.dropna().unique()
    
    return table, symbl  # return table and list of found market symbols

In [4]:
# read and preprocess shares from yahoo on current date
def read_shares(symbl, start, end):

    # set date interval
    start = end = get_date()

    # download into dataPanel
    shares_panel = web.get_data_yahoo(list(symbl), start, end)
 
    # convert Panel to the Multyindexed frame
    shares = shares_panel.to_frame()

    # remove index level 0 (date)
    shares.index = shares.index.droplevel(0)

    # convert index into column
    shares.reset_index(inplace=True)

    # rename `minor` column
    shares.rename(columns={'minor': 'SYMBOL'}, inplace=True)

    return shares

In [5]:
# create output xls from table and shares dataframes
def create_output(table, shares):
    
    # remove unnesessary columns from shares
    columns_to_drop = ['Close', 'High', 'Low', 'Open', 'Volume']
    shares.drop(columns_to_drop, axis=1, inplace=True)
    
    # create table
    table = pd.merge(left=table, right=shares, left_on='SYMBOL', right_on='SYMBOL')
    
     # reformat prices to 2 dec.digits
    table['Adj Close'] = table['Adj Close'].map(lambda x: round(x, 2))
    
    # rename some columns
    new_col_names = {'STOCKS':'STOCKS-BUY','Unnamed: 10':'STOCKS-SELL',
                'CALLS':'CALLS-BUY','Unnamed: 12': 'CALLS-SELL',
                'PUTTES':'PUTTES-BUY','Unnamed: 14': 'PUTTES-SELL'}
    table.rename(columns=new_col_names, inplace=True)
    
    # write new xls table
    table.to_excel('result_' + file_name, index=False)


### Start here

In [6]:
# main part
import os
import pandas as pd
#import pandas_datareader.data as web
from pandas_datareader import data as web

import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
#data = web.get_data_yahoo('AAPL', start="2018-05-10", end="2018-05-10")
from datetime import date
from datetime import datetime
import datetime
from datetime import timedelta

In [8]:
# works chain
file_name = choose_file()
start = end = get_date()
table, symbl = create_table(file_name)  # create pandas df
print ('\nwork tables created...')
shares = read_shares(symbl, start, end)  # deliver shares from yahoo
print ('shares delivered...')
create_output(table, shares)  # create, reformat and write output xls
print ('output file created')



work tables created...
[*********************100%***********************]  24 of 24 downloaded
shares delivered...
output file created


In [10]:
shares

,SYMBOL,Adj Close,Close,High,Low,Open,Volume
0,AAPL,189.309998,190.039993,190.369995,187.649994,187.740005,27989300.0
1,AGN,151.910004,151.910004,153.759995,147.000000,147.149994,3944800.0
2,AMZN,1609.079956,1609.079956,1615.599976,1603.439941,1608.479980,2817500.0
3,BTI,52.680000,52.680000,52.880001,52.330002,52.540001,1658500.0
4,BX,31.879999,31.879999,32.090000,31.690001,31.940001,2668200.0
5,CASH,106.599998,106.599998,107.050003,104.150002,104.750000,60200.0
6,DISH,31.780001,31.780001,31.860001,30.950001,31.340000,2711700.0
7,GE,14.690000,14.690000,14.760000,14.500000,14.760000,37480900.0
8,GM,37.160000,37.160000,37.220001,36.450001,36.459999,6814700.0
9,INCY,65.269997,65.269997,65.760002,64.529999,64.550003,3171800.0


### Sandbox space

In [13]:
shares.head(3)

,SYMBOL,Adj Close,Close,High,Low,Open,Volume
0,AAPL,189.309998,190.039993,190.369995,187.649994,187.740005,27989300.0
1,AGN,151.910004,151.910004,153.759995,147.000000,147.149994,3944800.0
2,AMZN,1609.079956,1609.079956,1615.599976,1603.439941,1608.479980,2817500.0


In [15]:
columns_to_drop = ['Close', 'High', 'Low', 'Open', 'Volume']
shares_1 = shares.drop(columns_to_drop, axis=1)

In [21]:
table_1 = pd.merge(left=table, right=shares_1, left_on='SYMBOL', right_on='SYMBOL')

In [22]:
table_1.head(3)

,TRX Date,SYMBOL,Expir Date,TRX CODE,Quantity,BASIS COST,Strike Price,Call Premium,Market at TRX,STOCKS,Unnamed: 10,CALLS,Unnamed: 12,PUTTES,Unnamed: 14,CASH,DIVDNT,Gain/ (Loss),BALANCE,Adj Close
0,NaT,CASH,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1853935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.599998
1,NaT,AAPL,NaT,NaN,9000.0,174.48,NaN,NaN,NaN,174.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189.309998
2,2017-12-22,AAPL,2018-02-16,S,20.0,2.39,185.0,NaN,NaN,175.01,4779.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189.309998


In [34]:
new_col_names = {'STOCKS':'STOCKS-BUY','Unnamed: 10':'STOCKS-SELL',
                'CALLS':'CALLS-BUY','Unnamed: 12': 'CALLS-SELL',
                'PUTTES':'PUTTES-BUY','Unnamed: 14': 'PUTTES-SELL'}

In [38]:
table_1.rename(columns=new_col_names, inplace=True)

In [39]:
table_1.to_excel('result_' + file_name, index=False)

In [ ]:
shares_panel = web.get_data_yahoo(list(symbl), start, end)

In [13]:
f = web.DataReader('F', 'morningstar', start, end)

NotImplementedError: data_source='morningstar' is not implemented

In [66]:
f

Close   High    Low   Open    Volume
Symbol Date                                            
F      2018-05-09  11.06  11.28  11.02  11.21  43731958
       2018-05-10  11.21  11.25  11.02  11.04  31519940

In [187]:
shares

,symbl,Adj Close,Close,High,Low,Open,Current Date
0,ABT,61.720001,61.720001,64.160004,60.740002,61.000000,2018-01-24
1,BX,36.080002,36.080002,36.330002,35.680000,36.250000,2018-01-24
2,CMCSA,42.990002,42.990002,44.000000,41.570000,42.700001,2018-01-24
3,CVX,131.389999,131.389999,132.669998,131.070007,131.380005,2018-01-24
4,MDT,85.760002,85.760002,86.129997,85.160004,85.830002,2018-01-24
5,MS,57.240002,57.240002,57.730000,56.900002,57.209999,2018-01-24
6,PM,110.599998,110.599998,110.889999,109.559998,109.559998,2018-01-24
7,SUN,32.540001,32.540001,33.040001,32.419998,32.529999,2018-01-24
8,UN,58.240002,58.240002,58.320000,57.990002,58.279999,2018-01-24


In [14]:
symbl

array([u'CMCSA', u'SUN', u'CVX', u'MDT', u'ABT', u'BX', u'PM', u'UN', u'MS'], dtype=object)

In [163]:
panel = web.DataReader(symbl, 'yahoo', start_date, end_date)

In [114]:
aaa.index.levels[0][0]

Timestamp('2018-01-24 00:00:00')

In [177]:
aaa.index.names

FrozenList([u'Date', u'minor'])

In [188]:
aaa.drop(start_date - timedelta(days=1) , level=0, axis=0)

Adj Close       Close        High         Low        Open  \
Date       minor                                                               
2018-01-24 ABT     61.720001   61.720001   64.160004   60.740002   61.000000   
           BX      36.080002   36.080002   36.330002   35.680000   36.250000   
           CMCSA   42.990002   42.990002   44.000000   41.570000   42.700001   
           CVX    131.389999  131.389999  132.669998  131.070007  131.380005   
           MDT     85.760002   85.760002   86.129997   85.160004   85.830002   
           MS      57.240002   57.240002   57.730000   56.900002   57.209999   
           PM     110.599998  110.599998  110.889999  109.559998  109.559998   
           SUN     32.540001   32.540001   33.040001   32.419998   32.529999   
           UN      58.240002   58.240002   58.320000   57.990002   58.279999   

                      Volume  
Date       minor              
2018-01-24 ABT    11704900.0  
           BX      5629000.0  
           CMCSA  48918800.0  
           CVX     6868200.0  
           MDT     5680200.0  
           MS      8403600.0  
           PM      4404100.0  
           SUN      490200.0  
           UN      1281200.0

In [149]:
from datetime import timedelta

In [151]:
start_date - timedelta(days=1)

datetime.date(2018, 1, 23)

In [173]:
start_date - timedelta(days=1)

datetime.date(2018, 1, 23)

In [4]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-05-09,186.550003,187.399994,185.220001,187.360001,186.640305,23211200


In [56]:
data.columns

AttributeError: 'Panel' object has no attribute 'columns'